In [1]:
import ipywidgets as ipw
import subprocess

In [2]:
from utils.control import *

In [3]:
msg,updates_needed,config = check_for_updates()

In [4]:
updates_needed

{'ssh_config': {'rename': True, 'hosts': ['daint.alps']},
 'codes': {'cp2k': {'rename': '5'}}}

In [5]:
if 'ssh_config' in updates_needed:
    update_ssh_config(config_path,config['ssh_config'],'cpi',rename=updates_needed['ssh_config']['rename'])

✅ Renamed /home/jovyan/.ssh/config → /home/jovyan/.ssh/202502191912_config
✅ Created new SSH config at /home/jovyan/.ssh/config


In [6]:
if 'ssh_config' in updates_needed:
    set_ssh(config['computers'],updates_needed['ssh_config']['hosts'])

🔄 Adding ela.cscs.ch to known_hosts...
🔄 Adding daint.alps.cscs.ch via ela.cscs.ch to known_hosts...
